In [8]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent  # Notebooks are in notebooks/, go up one level
sys.path.insert(0, str(project_root))

# Add venv packages
venv_path = project_root / "venv/lib/python3.13/site-packages"
if venv_path.exists():
    sys.path.insert(0, str(venv_path))

import numpy as np
import polars as pl
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set matplotlib backend for notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['figure.figsize'] = (12, 6)

print(f"✅ Project root: {project_root}")
print(f"✅ Src exists: {(project_root / 'src').exists()}")
print(f"✅ Matplotlib version: {mpl.__version__}")

✅ Project root: /Users/sudip/hull_tactical_market_prediction_using_hyperopt
✅ Src exists: True
✅ Matplotlib version: 3.10.7


In [9]:
# --- Import your modules ---
from src.features import create_advanced_features, select_features
from src.models.elastic_net import ElasticNetModel
from src.models.lightgbm_model import LightGBMModel
from src.models.xgboost_model import XGBoostModel
from src.models.ensemble import EnsembleModel
from evaluation import score

In [10]:
# --- Visualize Results ---

def plot_model_predictions(signals_dict):
    """Plot predictions from all models."""
    fig, axes = plt.subplots(len(signals_dict), 1, figsize=(14, 4*len(signals_dict)))
    
    if len(signals_dict) == 1:
        axes = [axes]
    
    for i, (name, signals) in enumerate(signals_dict.items()):
        axes[i].plot(signals, marker='o', linewidth=2, label=name)
        axes[i].set_title(f'{name} Predictions', fontsize=14, fontweight='bold')
        axes[i].set_xlabel('Test Sample Index')
        axes[i].set_ylabel('Signal Value')
        axes[i].axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Neutral')
        axes[i].axhspan(0, 2, alpha=0.1, color='green', label='Valid Range')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('artifacts/predictions_comparison.png', dpi=300, bbox_inches='tight')
    print("✅ Saved predictions plot to artifacts/predictions_comparison.png")
    plt.show()

def plot_feature_importance(feature_importance, top_n=20):
    """Plot feature importance."""
    sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:top_n]
    features, importances = zip(*sorted_features)
    
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(features)), importances, color='steelblue')
    plt.yticks(range(len(features)), features)
    plt.xlabel('Importance Score', fontsize=12)
    plt.title(f'Top {top_n} Feature Importance (LightGBM)', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    
    # Ensure artifacts directory exists
    Path('artifacts').mkdir(exist_ok=True)
    plt.savefig('artifacts/feature_importance.png', dpi=300, bbox_inches='tight')
    print(f"✅ Saved feature importance plot to artifacts/feature_importance.png")
    plt.show()

def plot_model_comparison(signals_dict):
    """Compare models side by side."""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Predictions comparison
    for name, signals in signals_dict.items():
        axes[0].plot(signals, marker='o', label=name, linewidth=2, markersize=8)
    axes[0].set_title('Model Predictions Comparison', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Test Sample Index')
    axes[0].set_ylabel('Signal Value')
    axes[0].axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Neutral')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Signal distribution
    for name, signals in signals_dict.items():
        axes[1].hist(signals, alpha=0.6, label=name, bins=20)
    axes[1].set_title('Signal Distribution', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Signal Value')
    axes[1].set_ylabel('Frequency')
    axes[1].axvline(x=1.0, color='r', linestyle='--', alpha=0.5, label='Neutral')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    Path('artifacts').mkdir(exist_ok=True)
    plt.savefig('artifacts/model_comparison.png', dpi=300, bbox_inches='tight')
    print("✅ Saved comparison plot to artifacts/model_comparison.png")
    plt.show()


In [11]:
def load_data(data_path: Path):
    """Load and preprocess data."""
    print("Loading data...")
    
    train = (
        pl.read_csv(data_path / "train.csv")
        .rename({'market_forward_excess_returns': 'target'})
        .with_columns(pl.exclude('date_id').cast(pl.Float64, strict=False))
        .head(-10)
    )
    
    test = (
        pl.read_csv(data_path / "test.csv")
        .rename({'lagged_forward_returns': 'target'})
        .with_columns(pl.exclude('date_id').cast(pl.Float64, strict=False))
    )
    
    print(f"Train shape: {train.shape}, Test shape: {test.shape}")
    return train, test

In [12]:
def create_features(df: pl.DataFrame) -> pl.DataFrame:
    """Create features from raw data."""
    print("Creating features...")
    
    vars_to_keep = ["S2", "E2", "E3", "P9", "S1", "S5", "I2", "P8", "P10", "P12", "P13"]
    
    # Create basic features
    df = df.with_columns([
        (pl.col("I2") - pl.col("I1")).alias("U1") if all(c in df.columns for c in ["I2", "I1"]) else pl.lit(None),
        (pl.col("M11") / ((pl.col("I2") + pl.col("I9") + pl.col("I7")) / 3)).alias("U2") if all(c in df.columns for c in ["M11", "I2", "I9", "I7"]) else pl.lit(None)
    ])
    
    df = df.select(["date_id", "target"] + vars_to_keep + ["U1", "U2"])
    
    # Fill nulls
    feature_cols = [col for col in df.columns if col not in ['date_id', 'target']]
    df = df.with_columns([
        pl.col(col).fill_null(pl.col(col).ewm_mean(com=0.5))
        for col in feature_cols
    ])
    
    # Create advanced features
    df = create_advanced_features(df, feature_cols)
    
    return df.drop_nulls()

In [13]:
def prepare_data(train: pl.DataFrame, test: pl.DataFrame):
    """Prepare train and test datasets."""
    print("Preparing data...")
    
    # Join and create features
    common_columns = [col for col in train.columns if col in test.columns]
    df = pl.concat([train.select(common_columns), test.select(common_columns)], how="vertical")
    
    df = create_features(df)
    
    # Split back
    train_date_ids = set(train.get_column('date_id'))
    test_date_ids = set(test.get_column('date_id'))
    
    train_df = df.filter(pl.col('date_id').is_in(list(train_date_ids)))
    test_df = df.filter(pl.col('date_id').is_in(list(test_date_ids)))
    
    # Get features
    feature_cols = select_features(train_df, method='all')
    print(f"Number of features: {len(feature_cols)}")
    
    # Prepare X and y
    X_train = train_df.select(feature_cols).to_numpy()
    y_train = train_df.get_column('target').to_numpy()
    
    X_test = test_df.select(feature_cols).to_numpy()
    y_test = test_df.get_column('target').to_numpy()
    
    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, y_train, X_test, y_test, scaler, feature_cols, test_df

In [14]:
def evaluate_model(name: str, predictions: np.ndarray, test_data: pl.DataFrame):
    """Evaluate model performance."""
    print(f"\nEvaluating {name}...")
    
    # Clip predictions to [0, 2] range
    signals = np.clip(predictions * 400 + 1, 0.0, 2.0)
    
    print(f"Signal range: [{signals.min():.4f}, {signals.max():.4f}]")
    print(f"Signal mean: {signals.mean():.4f}")
    
    return signals

In [15]:
def train_all_models(X_train, y_train, X_test, optimize=True):
    """Train all models and return predictions."""
    results = {}
    
    # ElasticNet
    print("\n" + "="*50)
    print("Training ElasticNet...")
    print("="*50)
    enet = ElasticNetModel(n_trials=20 if optimize else 0)
    enet.fit(X_train, y_train, optimize=optimize)
    results['ElasticNet'] = (enet, enet.predict(X_test))
    
    # LightGBM
    print("\n" + "="*50)
    print("Training LightGBM...")
    print("="*50)
    lgbm = LightGBMModel(n_trials=20 if optimize else 0)
    lgbm.fit(X_train, y_train, optimize=optimize)
    results['LightGBM'] = (lgbm, lgbm.predict(X_test))
    
    # XGBoost
    print("\n" + "="*50)
    print("Training XGBoost...")
    print("="*50)
    xgb_model = XGBoostModel(n_trials=20 if optimize else 0)
    xgb_model.fit(X_train, y_train, optimize=optimize)
    results['XGBoost'] = (xgb_model, xgb_model.predict(X_test))
    
    # Ensemble (simple average)
    print("\n" + "="*50)
    print("Training Ensemble...")
    print("="*50)
    ensemble_models = [enet, lgbm, xgb_model]
    ensemble = EnsembleModel(ensemble_models, weights=[0.3, 0.35, 0.35])
    ensemble.fit(X_train, y_train, optimize=False)
    results['Ensemble'] = (ensemble, ensemble.predict(X_test))
    
    return results

In [16]:
def main():
    """Main training function."""
    print("="*70)
    print("Hull Tactical Market Prediction - Advanced Training")
    print("="*70)
    
    # Paths
    data_path = Path('/Users/sudip/hull_tactical_market_prediction_using_hyperopt/input/hull-tactical-market-prediction/')
    
    # Load data
    train, test = load_data(data_path)
    
    # Prepare data
    X_train, y_train, X_test, y_test, scaler, feature_cols, test_df = prepare_data(train, test)
    
    print(f"\nData shapes:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
    
    # Train models
    results = train_all_models(X_train, y_train, X_test, optimize=True)
    
    # Evaluate models
    print("\n" + "="*70)
    print("Model Evaluation Results")
    print("="*70)
    
    all_signals = {}
    for name, (model, predictions) in results.items():
        signals = evaluate_model(name, predictions, test_df)
        all_signals[name] = signals
    
    # Feature importance
    print("\n" + "="*70)
    print("Feature Importance (LightGBM)")
    print("="*70)
    try:
        lgbm_model = results['LightGBM'][0]
        feature_importance = lgbm_model.get_feature_importance()
        # Sort by importance
        sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
        for i, (feature, importance) in enumerate(sorted_features[:10]):
            print(f"{i+1}. {feature}: {importance:.4f}")
    except Exception as e:
        print(f"Could not get feature importance: {e}")
    
    return results, all_signals, test_df

In [17]:
if __name__ == "__main__":
    results, signals, test_data = main()
    print("\nTraining completed successfully!")

[I 2025-10-27 19:50:04,744] A new study created in memory with name: no-name-2ba7a6de-46e7-4915-8adf-6834c9644df0


Hull Tactical Market Prediction - Advanced Training
Loading data...
Train shape: (8980, 98), Test shape: (10, 99)
Preparing data...
Creating features...
Number of features: 88

Data shapes:
X_train: (7469, 88), y_train: (7469,)
X_test: (10, 88), y_test: (10,)

Training ElasticNet...


Best trial: 0. Best value: 0.0109072:  25%|██▌       | 5/20 [00:00<00:00, 55.96it/s]

[I 2025-10-27 19:50:04,785] Trial 0 finished with value: 0.010907207084948136 and parameters: {'alpha': 0.0003953389548788279, 'l1_ratio': 0.61318554714667}. Best is trial 0 with value: 0.010907207084948136.
[I 2025-10-27 19:50:04,796] Trial 1 finished with value: 0.01092168130280988 and parameters: {'alpha': 0.3768202553070682, 'l1_ratio': 0.7489830158071639}. Best is trial 0 with value: 0.010907207084948136.
[I 2025-10-27 19:50:04,807] Trial 2 finished with value: 0.01092168130280988 and parameters: {'alpha': 0.04389262669466367, 'l1_ratio': 0.22087591331934953}. Best is trial 0 with value: 0.010907207084948136.
[I 2025-10-27 19:50:04,816] Trial 3 finished with value: 0.010921140284348475 and parameters: {'alpha': 0.004631255456605109, 'l1_ratio': 0.20689971701343612}. Best is trial 0 with value: 0.010907207084948136.
[I 2025-10-27 19:50:04,829] Trial 4 finished with value: 0.010912792197448885 and parameters: {'alpha': 0.0028398281576175557, 'l1_ratio': 0.1981423329768207}. Best is 

Best trial: 9. Best value: 0.010906:  45%|████▌     | 9/20 [00:00<00:00, 12.51it/s] 

...

[I 2025-10-27 19:51:03,704] Trial 17 finished with value: 0.009701030499688044 and parameters: {'max_depth': 8, 'learning_rate': 0.015330675671653402, 'subsample': 0.885269875181822, 'colsample_bytree': 0.6477985145166647, 'colsample_bylevel': 0.6753741278216345, 'min_child_weight': 9, 'gamma': 1.2618233510181862e-05, 'reg_alpha': 0.28115387056101465, 'reg_lambda': 1.7342540349920564e-05}. Best is trial 17 with value: 0.009701030499688044.


Best trial: 17. Best value: 0.00970103:  95%|█████████▌| 19/20 [00:25<00:02,  2.64s/it]

[I 2025-10-27 19:51:07,772] Trial 18 finished with value: 0.01004541075189271 and parameters: {'max_depth': 10, 'learning_rate': 0.014640901402492967, 'subsample': 0.909314232448642, 'colsample_bytree': 0.6579400841848639, 'colsample_bylevel': 0.6426160885872951, 'min_child_weight': 9, 'gamma': 1.6242720306667466e-05, 'reg_alpha': 0.0001851902793440002, 'reg_lambda': 2.2818319706598933e-05}. Best is trial 17 with value: 0.009701030499688044.


Best trial: 17. Best value: 0.00970103: 100%|██████████| 20/20 [00:28<00:00,  1.41s/it]


[I 2025-10-27 19:51:10,731] Trial 19 finished with value: 0.00996994111861836 and parameters: {'max_depth': 8, 'learning_rate': 0.016038245911343365, 'subsample': 0.9116228740119612, 'colsample_bytree': 0.49541449159042805, 'colsample_bylevel': 0.7565365804364317, 'min_child_weight': 9, 'gamma': 1.5346791636862368e-07, 'reg_alpha': 0.03772486026804628, 'reg_lambda': 0.001022024144451998}. Best is trial 17 with value: 0.009701030499688044.
Best XGBoost score: 0.009701
Best XGBoost params: {'max_depth': 8, 'learning_rate': 0.015330675671653402, 'subsample': 0.885269875181822, 'colsample_bytree': 0.6477985145166647, 'colsample_bylevel': 0.6753741278216345, 'min_child_weight': 9, 'gamma': 1.2618233510181862e-05, 'reg_alpha': 0.28115387056101465, 'reg_lambda': 1.7342540349920564e-05}

Training Ensemble...

Model Evaluation Results

Evaluating ElasticNet...
Signal range: [0.8733, 1.1569]
Signal mean: 0.9953

Evaluating LightGBM...
Signal range: [0.0000, 2.0000]
Signal mean: 1.0465

Evaluatin